In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split


from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score

C:\Users\felip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base = pd.read_csv("basePreProcessedAllAbFinal_com_NaN.csv")
y_data = base["status"]
x_data = base.drop(["status"], axis=1)

attrCount = x_data.shape[1]

In [3]:
random_state = 1
folds = 10
countFold = 1

In [4]:
finalResults = {"Model": [], "ACC-Balanced": [], "F1-Macro": [], "AUC": [], "MCC": []}

In [ ]:
#RANDOM FOREST
classifier = RandomForestClassifier(random_state=random_state, criterion="gini", max_features=13, n_estimators=1275, max_depth=32) #RF-PipeNone

#classifier = RandomForestClassifier(random_state=random_state, criterion="log_loss", max_features=7, n_estimators=1375, max_depth=62) #RF-Pipe-SCALER+PCA+SMOTE

#classifier = RandomForestClassifier(random_state=random_state, criterion="entropy", max_features=7, n_estimators=600, max_depth=27) #RF-Pipe-SCALER+SMOTE

#classifier = RandomForestClassifier(random_state=random_state, criterion="gini", max_features=7, n_estimators=900, max_depth=97) #RF-Pipe-IMPUTER+SCALER+PCA+SMOTE

#classifier = RandomForestClassifier(random_state=random_state, criterion="gini", max_features=17, n_estimators=775, max_depth=82) #RF-Pipe-IMPUTER+SCALER+SMOTE

#classifier = RandomForestClassifier(random_state=random_state, criterion="log_loss", max_features=13, n_estimators=1350, max_depth=47) #RF-Pipe-KNN_IMPUTER+SCALER+SMOTE

#XGBoost

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.05, n_estimators=300, max_depth=8, subsample=0.8, colsample_bytree=0.5, gamma=0.4, min_child_weight=5) #XGB-PipeNone

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.025, n_estimators=450, max_depth=22, subsample=1.0, colsample_bytree=0.5, gamma=0.7000000000000001, min_child_weight=5) #XGB-Pipe-SCALER+PCA+SMOTE

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.05, n_estimators=425, max_depth=18, subsample=0.8, colsample_bytree=0.6000000000000001, gamma=0.30000000000000004, min_child_weight=3) #XGB-Pipe-SCALER+SMOTE

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.05, n_estimators=275, max_depth=18, subsample=0.8, colsample_bytree=0.7, gamma=0.5, min_child_weight=4) #XGB-Pipe-IMPUTER+SCALER+PCA+SMOTE

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.025, n_estimators=425, max_depth=8, subsample=0.9, colsample_bytree=0.5, gamma=1.3, min_child_weight=1 ) #XGB-Pipe-IMPUTER+SCALER+SMOTE

#classifier = XGBClassifier(random_state=random_state, learning_rate=0.025, n_estimators=250, max_depth=10, subsample=0.9, colsample_bytree=0.5, gamma=1.0, min_child_weight=2) #XGB-Pipe-KNN_IMPUTER+SCALER+SMOTE

#imputer = KNNImputer(n_neighbors=5)
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')
kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)

countFold = 1

results = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}

#Treinamendo do modelo com k-fold corss validation e aplicação do pipeline
for train_index, test_index in kf.split(base):
    
    x_train, x_test = x_data.iloc[train_index], x_data.iloc[test_index]
    y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]

    #Mean Imputation
    #x_train = imputer.fit_transform(x_train) #Uncomment this for pipeline 2
    #x_test = imputer.transform(x_test) #Uncomment this for pipeline 2

    # Standardize the sets using the same scaler
    #x_train = scaler.fit_transform(x_train)
    #x_test = scaler.transform(x_test)

    # Apply PCA to training data to retain 95% variance
    #pca = PCA(n_components=0.95) #Comment this for pipeline 1_semPCA
    #x_train = pca.fit_transform(x_train) #Comment this for pipeline 1_semPCA

    # Apply the same PCA transformation to the test set
    #x_test = pca.transform(x_test) #Comment this for pipeline 1_semPCA

    # SMOTE augmentation on the PCA-reduced training set
    #smote = SMOTE(k_neighbors=3, random_state=random_state)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    x_train = pd.DataFrame(x_train)
    x_test = pd.DataFrame(x_test)
    
    classifier.fit(x_train, y_train)
    y_pred_proba = classifier.predict_proba(x_test)[:, 1]
    y_pred = (y_pred_proba >= 0.5).astype(int) #Threshold de 0.5 considerado

    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    auc = roc_auc_score(y_test, y_pred_proba) #CORRIGIDO: y_pred -> y_pred_proba
    mcc = matthews_corrcoef(y_test, y_pred)

    results['ACC-Balanced'].append(balanced_acc)
    results['F1-Macro'].append(f1_macro)
    results['AUC'].append(auc)
    results['MCC'].append(mcc)

finalResults["Model"].append("RF-Pipe-None")
finalResults["ACC-Balanced"].append(np.average(results['ACC-Balanced']))
finalResults["F1-Macro"].append(np.average(results['F1-Macro']))
finalResults["AUC"].append(np.average(results['AUC']))
finalResults["MCC"].append(np.average(results['MCC']))

print(finalResults)

In [ ]:
resultsDf = pd.DataFrame(finalResults)
print(resultsDf)

In [ ]:
print("cuda") if torch.cuda.is_available() else print("cpu")
global device 
device = "cuda" if torch.cuda.is_available() else "cpu"

model = nn.Sequential(nn.Linear(attrCount, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Sigmoid()
                )


#Pipe None
#Trial 13 finished with value: 0.8877320532336895 and parameters: {'num_layers': 2, 'n_units_layer_0': 512, 'n_units_layer_1': 32, 'activation': 'ReLU', 'use_batch_norm': False, 'use_dropout': True, 'optimizer': 'AdamW', 'lr': 0.00012295920205037024, 'weight_decay': 6.268225479552331e-05}. Best is trial 13 with value: 0.8877320532336895.

#Pipe IMPUTER-SCALER-SMOTE
#Trial 67 finished with value: 0.9471169493848363 and parameters: {'num_layers': 2, 'n_units_layer_0': 512, 'n_units_layer_1': 256, 'activation': 'ReLU', 'use_batch_norm': False, 'use_dropout': False, 'optimizer': 'AdamW', 'lr': 0.00037632176070769957, 'weight_decay': 1.873329227021332e-05}. Best is trial 67 with value: 0.9471169493848363. Best trial: 67. Best value: 0.947117

# Modelo
model = model.to(device)

# Otimizador
optimizer = optim.AdamW(model.parameters(), lr=0.00037632176070769957, weight_decay=1.873329227021332e-05)

#Loss
loss_fn = nn.BCELoss().to(device)


#imputer = KNNImputer(n_neighbors=5) #Botar fora da erro
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)

countFold = 1
results = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}

#Treinamendo do modelo com k-fold corss validation e aplicação do pipeline
for train_index, test_index in kf.split(base):
    x_train, x_test = x_data.iloc[train_index], x_data.iloc[test_index]
    y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]

    print(f"Fold: {countFold}/{folds}")
    countFold += 1

    #Mean Imputation
    x_train = imputer.fit_transform(x_train) #Uncomment this for pipeline 2
    x_test = imputer.transform(x_test) #Uncomment this for pipeline 2

    # Standardize the sets using the same scaler
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    # Apply PCA to training data to retain 95% variance
    #pca = PCA(n_components=0.95) #Comment this for pipeline 1_semPCA
    #x_train = pca.fit_transform(x_train) #Comment this for pipeline 1_semPCA

    # Apply the same PCA transformation to the test set
    #x_test = pca.transform(x_test) #Comment this for pipeline 1_semPCA

    # SMOTE augmentation on the PCA-reduced training set
    smote = SMOTE(k_neighbors=3, random_state=random_state)
    x_train, y_train = smote.fit_resample(x_train, y_train)

    x_train = pd.DataFrame(x_train)
    x_test = pd.DataFrame(x_test)

    x_train = torch.tensor(x_train.to_numpy(), dtype=torch.float32)
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    x_test = torch.tensor(x_test.to_numpy(), dtype=torch.float32)
    y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

    # Treinamento
    num_epochs = 500
    model.to(device)

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=32, shuffle=False) #pin_memory=True - Para GPU
    val_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=32, shuffle=False) #pin_memory=True - Para GPU

    best_val_loss = float('inf')
    #best_val_accuracy = 0.0
    patience = 100
    counter = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Validação por epoca
        model.eval()
        val_loss = 0
        y_preds, y_true = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch).squeeze()
                loss = loss_fn(y_pred, y_batch)
                val_loss += loss.item()
                y_preds.append(y_pred.cpu())
                y_true.append(y_batch.cpu())

        val_loss /= len(val_loader)
        y_preds = torch.cat(y_preds).round()
        y_true = torch.cat(y_true)
        val_accuracy = balanced_accuracy_score(y_true, y_preds)

        # Early stopping check
        #if val_loss < best_val_loss or val_accuracy > best_val_accuracy:
            #best_val_loss = min(best_val_loss, val_loss)
            #best_val_accuracy = max(best_val_accuracy, val_accuracy)
            #counter = 0
        if val_loss < best_val_loss:
            best_val_loss = min(best_val_loss, val_loss)
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break
        #print(epoch)

    # Avaliação final do fold
    model.eval()
    y_preds, y_true = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).squeeze()
            y_preds.append(y_pred.cpu())
            y_true.append(y_batch.cpu())

    y_pred_proba = torch.cat(y_preds)
    y_pred = (y_pred_proba >= 0.5).int() #Threshold de 0.5 considerado
    y_test = torch.cat(y_true) #Mantido uso do y_true pois pode ser que os labels de teste não estejam na mesma posição das predições

    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    auc = roc_auc_score(y_test, y_pred_proba) #CORRIGIDO: y_pred -> y_pred_proba
    mcc = matthews_corrcoef(y_test, y_pred)

    results['ACC-Balanced'].append(balanced_acc)
    results['F1-Macro'].append(f1_macro)
    results['AUC'].append(auc)
    results['MCC'].append(mcc)

print(np.mean(results["ACC-Balanced"]))
print(np.mean(results["F1-Macro"]))
print(np.mean(results["AUC"]))
print(np.mean(results["MCC"]))

In [67]:
resultsDfMLP = pd.DataFrame(results)
print(resultsDfMLP)

   ACC-Balanced  F1-Macro       AUC       MCC
0      0.688047  0.682988  0.688047  0.375928
1      0.935781  0.935253  0.935781  0.870620
2      0.943191  0.942690  0.943191  0.885431
3      0.966542  0.966430  0.966542  0.932874
4      0.961616  0.961282  0.961616  0.922680
5      0.955020  0.953607  0.955020  0.909700
6      0.984608  0.984491  0.984608  0.969035
7      0.981785  0.981881  0.981785  0.963776
8      0.994784  0.994784  0.994784  0.989569
9      0.984272  0.984466  0.984272  0.968985


In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba) #CORRIGIDO: y_pred -> y_pred_proba
roc_auc = auc(fpr, tpr)

# Find the point corresponding to threshold = 0.5
threshold_05_idx = np.argmin(np.abs(thresholds - 0.5))

# Plot ROC curve
plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, color='darkblue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=1, label='Random Classifier')

# Mark threshold = 0.5 point
plt.scatter(fpr[threshold_05_idx], tpr[threshold_05_idx], color='red', s=100, edgecolors='black', label='Threshold = 0.5')

# Labels and formatting
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curve', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


#### Testes com holdout data

In [1]:
base = pd.read_csv("basePreProcessedAllAbFinal.csv")


NameError: name 'pd' is not defined

In [96]:
x_data_all = base.drop(["status"], axis=1)
y_data_all = base["status"]

attrCount = x_data.shape[1]

'''
base_reduced, holdout = train_test_split(base, test_size=0.1, random_state=random_state) 

x_test_holdout = holdout.drop(["status"], axis=1)
y_test_holdout = holdout["status"]

x_data = base_reduced.drop(["status"], axis=1)
y_data = base_reduced["status"]

attrCount = x_data.shape[1]
'''

'\nbase_reduced, holdout = train_test_split(base, test_size=0.1, random_state=random_state) \n\nx_test_holdout = holdout.drop(["status"], axis=1)\ny_test_holdout = holdout["status"]\n\nx_data = base_reduced.drop(["status"], axis=1)\ny_data = base_reduced["status"]\n\nattrCount = x_data.shape[1]\n'

In [ ]:
kf2 = KFold(n_splits=folds, shuffle=True, random_state=random_state)   
final_results_test = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}
final_results_holdout = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}

for base_reduced_index, holdout_index in kf.split(base):
    x_reduced, x_holdout = x_data_all.iloc[base_reduced_index], x_data_all.iloc[holdout_index]
    y_reduced, y_holdout = y_data_all.iloc[base_reduced_index], y_data_all.iloc[holdout_index]

    x_test_holdout = x_holdout
    y_test_holdout = y_holdout

    x_data = x_reduced
    y_data = y_reduced

    attrCount = x_data.shape[1]

    print("cuda") if torch.cuda.is_available() else print("cpu")
    global device 
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = nn.Sequential(nn.Linear(attrCount, 512),
                    nn.ReLU(),
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Linear(256, 1),
                    nn.Sigmoid()
                    )


    #Pipe None
    #Trial 13 finished with value: 0.8877320532336895 and parameters: {'num_layers': 2, 'n_units_layer_0': 512, 'n_units_layer_1': 32, 'activation': 'ReLU', 'use_batch_norm': False, 'use_dropout': True, 'optimizer': 'AdamW', 'lr': 0.00012295920205037024, 'weight_decay': 6.268225479552331e-05}. Best is trial 13 with value: 0.8877320532336895.

    #Pipe IMPUTER-SCALER-SMOTE
    #Trial 67 finished with value: 0.9471169493848363 and parameters: {'num_layers': 2, 'n_units_layer_0': 512, 'n_units_layer_1': 256, 'activation': 'ReLU', 'use_batch_norm': False, 'use_dropout': False, 'optimizer': 'AdamW', 'lr': 0.00037632176070769957, 'weight_decay': 1.873329227021332e-05}. Best is trial 67 with value: 0.9471169493848363. Best trial: 67. Best value: 0.947117

    # Modelo
    model = model.to(device)

    # Otimizador
    optimizer = optim.AdamW(model.parameters(), lr=0.00037632176070769957, weight_decay=1.873329227021332e-05)

    #Loss
    loss_fn = nn.BCELoss().to(device)


    #imputer = KNNImputer(n_neighbors=5) #Botar fora da erro
    imputer = SimpleImputer(strategy='mean')
    scaler = StandardScaler()
    kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)

    countFold = 1
    results = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}
    results_holdout = {'ACC-Balanced':[], 'F1-Macro':[], 'AUC': [], 'MCC': []}

    #Treinamendo do modelo com k-fold corss validation e aplicação do pipeline
    for train_index, test_index in kf.split(x_reduced, y_reduced):
        x_train, x_test = x_reduced.iloc[train_index], x_reduced.iloc[test_index]
        y_train, y_test = y_reduced.iloc[train_index], y_reduced.iloc[test_index]

        print(f"Fold: {countFold}/{folds}")
        countFold += 1

        #Mean Imputation
        x_train = imputer.fit_transform(x_train) #Uncomment this for pipeline 2
        x_test = imputer.transform(x_test) #Uncomment this for pipeline 2
        if countFold == 1:
            x_test_holdout = imputer.transform(x_test_holdout)
        
        # Standardize the sets using the same scaler
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        if countFold == 1:
            x_test_holdout = scaler.transform(x_test_holdout)

        # Apply PCA to training data to retain 95% variance
        #pca = PCA(n_components=0.95) #Comment this for pipeline 1_semPCA
        #x_train = pca.fit_transform(x_train) #Comment this for pipeline 1_semPCA

        # Apply the same PCA transformation to the test set
        #x_test = pca.transform(x_test) #Comment this for pipeline 1_semPCA

        # SMOTE augmentation on the PCA-reduced training set
        smote = SMOTE(k_neighbors=3, random_state=random_state)
        x_train, y_train = smote.fit_resample(x_train, y_train)

        x_train = pd.DataFrame(x_train)
        x_test = pd.DataFrame(x_test)
        x_test_holdout = pd.DataFrame(x_test_holdout)

        x_train = torch.tensor(x_train.to_numpy(), dtype=torch.float32)
        y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
        x_test = torch.tensor(x_test.to_numpy(), dtype=torch.float32)
        y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

        if countFold == 1:
            x_test_holdout = torch.tensor(x_test_holdout.to_numpy(), dtype=torch.float32)
            y_test_holdout = torch.tensor(y_test_holdout.to_numpy(), dtype=torch.float32)

        # Treinamento
        num_epochs = 500
        model.to(device)

        train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=32, shuffle=False) #pin_memory=True - Para GPU
        val_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=32, shuffle=False) #pin_memory=True - Para GPU
        if countFold == 1:
            holdout_loader = DataLoader(TensorDataset(x_test_holdout, y_test_holdout), batch_size=32, shuffle=False) #pin_memory=True - Para GPU

        best_val_loss = float('inf')
        #best_val_accuracy = 0.0
        patience = 100
        counter = 0

        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                y_pred = model(X_batch).squeeze()
                loss = loss_fn(y_pred, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            # Validação por epoca
            model.eval()
            val_loss = 0
            y_preds, y_true = [], []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    y_pred = model(X_batch).squeeze()
                    loss = loss_fn(y_pred, y_batch)
                    val_loss += loss.item()
                    y_preds.append(y_pred.cpu())
                    y_true.append(y_batch.cpu())

            val_loss /= len(val_loader)
            y_preds = torch.cat(y_preds).round()
            y_true = torch.cat(y_true)
            val_accuracy = balanced_accuracy_score(y_true, y_preds)

            # Early stopping check
            #if val_loss < best_val_loss or val_accuracy > best_val_accuracy:
                #best_val_loss = min(best_val_loss, val_loss)
                #best_val_accuracy = max(best_val_accuracy, val_accuracy)
                #counter = 0
            if val_loss < best_val_loss:
                best_val_loss = min(best_val_loss, val_loss)
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early stopping triggered at epoch {epoch}")
                    break
            #print(epoch)

        # Avaliação final do fold
        model.eval()
        y_preds, y_true = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch).squeeze()
                y_preds.append(y_pred.cpu())
                y_true.append(y_batch.cpu())

        y_pred_proba = torch.cat(y_preds)
        y_pred = (y_pred_proba >= 0.5).int() #Threshold de 0.5 considerado
        y_test = torch.cat(y_true) #Mantido uso do y_true pois pode ser que os labels de teste não estejam na mesma posição das predições

        balanced_acc = balanced_accuracy_score(y_test, y_pred) #Se quisermos maximizar a acurácia balanceada
        f1_macro = f1_score(y_test, y_pred, average='macro') #Se quisermos maximizar o F1 macro
        auc = roc_auc_score(y_test, y_pred)
        mcc = matthews_corrcoef(y_test, y_pred) #Se quisermos maximizar o MCC

        results['ACC-Balanced'].append(balanced_acc)
        results['F1-Macro'].append(f1_macro)
        results['AUC'].append(auc)
        results['MCC'].append(mcc)

        # Avaliação final do fold com holdout
        model.eval()
        y_preds, y_true = [], []
        with torch.no_grad():
            for X_batch, y_batch in holdout_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch).squeeze()
                y_preds.append(y_pred.cpu())
                y_true.append(y_batch.cpu())

        y_pred_proba = torch.cat(y_preds)
        y_pred = (y_pred_proba >= 0.5).int() #Threshold de 0.5 considerado
        y_test = torch.cat(y_true) #Mantido uso do y_true pois pode ser que os labels de teste não estejam na mesma posição das predições

        balanced_acc = balanced_accuracy_score(y_test, y_pred) #Se quisermos maximizar a acurácia balanceada
        f1_macro = f1_score(y_test, y_pred, average='macro') #Se quisermos maximizar o F1 macro
        auc = roc_auc_score(y_test, y_pred)
        mcc = matthews_corrcoef(y_test, y_pred) #Se quisermos maximizar o MCC

        results_holdout['ACC-Balanced'].append(balanced_acc)
        results_holdout['F1-Macro'].append(f1_macro)
        results_holdout['AUC'].append(auc)
        results_holdout['MCC'].append(mcc)

    final_results_test['ACC-Balanced'].append(np.mean(results["ACC-Balanced"]))
    final_results_test['F1-Macro'].append(np.mean(results["F1-Macro"]))
    final_results_test['AUC'].append(np.mean(results["AUC"]))
    final_results_test['MCC'].append(np.mean(results["MCC"]))

    final_results_holdout['ACC-Balanced'].append(np.mean(results_holdout["ACC-Balanced"]))
    final_results_holdout['F1-Macro'].append(np.mean(results_holdout["F1-Macro"]))
    final_results_holdout['AUC'].append(np.mean(results_holdout["AUC"]))
    final_results_holdout['MCC'].append(np.mean(results_holdout["MCC"]))

print(np.mean(final_results_test["ACC-Balanced"]))
print(np.mean(final_results_test["F1-Macro"]))
print(np.mean(final_results_test["AUC"]))
print(np.mean(final_results_test["MCC"]))

print(np.mean(final_results_holdout["ACC-Balanced"]))
print(np.mean(final_results_holdout["F1-Macro"]))
print(np.mean(final_results_holdout["AUC"]))
print(np.mean(final_results_holdout["MCC"]))


cpu
Fold: 1/10
Early stopping triggered at epoch 116
Fold: 2/10
Early stopping triggered at epoch 112
Fold: 3/10
Early stopping triggered at epoch 108
Fold: 4/10
Early stopping triggered at epoch 104
Fold: 5/10
Early stopping triggered at epoch 119
Fold: 6/10
Early stopping triggered at epoch 103
Fold: 7/10
Early stopping triggered at epoch 171
Fold: 8/10
Early stopping triggered at epoch 122
Fold: 9/10
Early stopping triggered at epoch 117
Fold: 10/10
Early stopping triggered at epoch 151
cpu
Fold: 1/10
Early stopping triggered at epoch 112
Fold: 2/10
Early stopping triggered at epoch 108
Fold: 3/10
Early stopping triggered at epoch 105
Fold: 4/10
Early stopping triggered at epoch 106
Fold: 5/10
Early stopping triggered at epoch 108
Fold: 6/10
Early stopping triggered at epoch 101
Fold: 7/10
Early stopping triggered at epoch 105
Fold: 8/10
Early stopping triggered at epoch 110
Fold: 9/10
Early stopping triggered at epoch 146
Fold: 10/10
Early stopping triggered at epoch 103
cpu
Fold: 

In [93]:
final_resultsDf = pd.DataFrame(final_results_test)
print(final_resultsDf)

final_resultsDfHoldout = pd.DataFrame(final_results_holdout)
print(final_resultsDfHoldout)

   ACC-Balanced  F1-Macro       AUC       MCC
0      0.904158  0.898154  0.904158  0.816054
1      0.941055  0.939996  0.941055  0.882885
2      0.920377  0.918007  0.920377  0.844110
3      0.932255  0.929812  0.932255  0.866106
4      0.939748  0.938980  0.939748  0.879333
5      0.929756  0.924411  0.929756  0.863294
6      0.946311  0.945803  0.946311  0.892049
7      0.939702  0.938850  0.939702  0.878313
8      0.942451  0.941105  0.942451  0.885126
9      0.928446  0.926965  0.928446  0.858112
   ACC-Balanced  F1-Macro       AUC       MCC
0      0.894046  0.889901  0.894046  0.797220
1      0.947808  0.947570  0.947808  0.895681
2      0.901687  0.901342  0.901687  0.810459
3      0.916386  0.915634  0.916386  0.835846
4      0.942204  0.942299  0.942204  0.885292
5      0.927884  0.923464  0.927884  0.863646
6      0.934368  0.935638  0.934368  0.871974
7      0.941685  0.941817  0.941685  0.884081
8      0.935569  0.935125  0.935569  0.870911
9      0.911199  0.909537  0.91119

0.9324260221382561
0.9302083982133744
0.9324260221382561
0.866538253091449
0.9252836907917636
0.9242326753623697
0.9252836907917636
0.8537761096948471

Fold   ACC-Balanced  F1-Macro       AUC       MCC
0      0.904158  0.898154  0.904158  0.816054
1      0.941055  0.939996  0.941055  0.882885
2      0.920377  0.918007  0.920377  0.844110
3      0.932255  0.929812  0.932255  0.866106
4      0.939748  0.938980  0.939748  0.879333
5      0.929756  0.924411  0.929756  0.863294
6      0.946311  0.945803  0.946311  0.892049
7      0.939702  0.938850  0.939702  0.878313
8      0.942451  0.941105  0.942451  0.885126
9      0.928446  0.926965  0.928446  0.858112
Fold   ACC-Balanced  F1-Macro       AUC       MCC
0      0.894046  0.889901  0.894046  0.797220
1      0.947808  0.947570  0.947808  0.895681
2      0.901687  0.901342  0.901687  0.810459
3      0.916386  0.915634  0.916386  0.835846
4      0.942204  0.942299  0.942204  0.885292
5      0.927884  0.923464  0.927884  0.863646
6      0.934368  0.935638  0.934368  0.871974
7      0.941685  0.941817  0.941685  0.884081
8      0.935569  0.935125  0.935569  0.870911
9      0.911199  0.909537  0.911199  0.822651